## Simple Mission in Guided-Mode

This notebook serves to get familiar with the simulator

In [1]:
import os

from config import (
    ARDU_LOGS_PATH,
    ARDUPILOT_VEHICLE_PATH,
    ENV_CMD_PYT,
    VEH_PARAMS_PATH,
    BasePort,
)
from helpers import clean, create_process, setup_logging, wait_for_port
from helpers.connections.mavlink.conn import create_tcp_conn
from plan import State
from plan.planner import Plan

clean()

## Launch Copter (ardupilot)

In [2]:
#This must agree with first waypoint in mission.waypoint
spawn_str= '-35.3633245,149.1652241,0.0,0' 

sim_vehicle_path = os.path.expanduser(ARDUPILOT_VEHICLE_PATH)
sysid = 1
vehicle_cmd = (
    f"python3 {sim_vehicle_path} "
    f"-v ArduCopter "
    f"-I{sysid-1} --sysid {sysid} "
    f"--no-rebuild "
    f"--use-dir={ARDU_LOGS_PATH} "
    f"--add-param-file {VEH_PARAMS_PATH} "
    f"--no-mavproxy "
    f"--port-offset=0 "  # ArduPilot automatically binds TCP port 5760 (BasePort.ARP)
    f'--custom-location={spawn_str}'
)

create_process(
                vehicle_cmd,
                after="exec bash",
                visible= True,
                suppress_output=False,
                title="ardu_vehicle",
                env_cmd=ENV_CMD_PYT,
            )
wait_for_port(BasePort.ARP, timeout=1)

Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...
Waiting for port 5760 to open...


## Connect to the vehicle

In [3]:
conn = create_tcp_conn(
    base_port=BasePort.ARP, 
    offset=0,
    role="client", 
    sysid=sysid
)

print("✅ TCP connection established!")

✅ TCP connection established!


## Create Plan in guided mode

Action commands are sent online, in contrast to auto mode where the full plan is loaded into the UAV before starting the mission.

In [4]:
plan = Plan.square(side_len=5, alt=5, wp_margin=0.5, navegation_speed=5)

In [5]:
plan

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EFKStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <TakeOff '🔹 take off to 1.0 m'>
  🕓 <Action '🛩️ FLY'>
    🕓 <GoTo '🔹 go to  -> (0, 0, 5)'>
    🕓 <GoTo '🔹 go to  -> (0, 5, 5)'>
    🕓 <GoTo '🔹 go to  -> (5, 5, 5)'>
    🕓 <GoTo '🔹 go to  -> (5, 0, 5)'>
    🕓 <GoTo '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Land '🔹 land'>

# Execute Plan in guided mode

In [ ]:
plan.bind(conn)
setup_logging('plan',verbose=2)
while plan.state != State.DONE:
    plan.act()

11:18:23 - plan - DEBUG - ▶️ Vehicle 1: Plan Started: 📋 Square Trajectory
11:18:23 - plan - DEBUG - ▶️ Vehicle 1: Action Started: 🔧 PREARM
11:18:23 - plan - DEBUG - ▶️ Vehicle 1: CheckDisarmed Started: Check disarmed
11:18:23 - plan - DEBUG - ✅ Vehicle 1: CheckDisarmed Done: Check disarmed
11:18:23 - plan - DEBUG - Vehicle 1: 📡 Requested message EKF_STATUS_REPORT at 1.00 Hz
11:18:23 - plan - DEBUG - ▶️ Vehicle 1: EFKStatus Started: Check EKF status
11:18:26 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
11:18:27 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: ATTITUDE, VELOCITY_HORIZ, POS_VERT_ABS, POS_HORIZ_ABS
11:18:28 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
11:18:29 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_ABS
11:18:30 - plan - DEBUG - 🛰️ Vehicle 1: Waiting for EKF to be ready... Pending: POS_HORIZ_A

In [ ]:
print(plan)

✅ <Plan '📋 Square Trajectory'>
  ✅ <Action '🔧 PREARM'>
    ✅ <CheckDisarmed '🔹 Check disarmed'>
    ✅ <EFKStatus '🔹 Check EKF status'>
    ✅ <GPSStatus '🔹 Check GPS'>
    ✅ <CheckSystem '🔹 Check system status'>
  ✅ <Action '⚙️  MODE: GUIDED'>
    ✅ <SwitchMode '🔹 Switch to GUIDED'>
  ✅ <Action '🔐 ARM'>
    ✅ <Arm '🔹 arm'>
  ✅ <Action '🛫 TAKEOFF'>
    ✅ <TakeOff '🔹 take off to 1.0 m'>
  ✅ <Action '🛩️ FLY'>
    ✅ <GoTo '🔹 go to  -> (0, 0, 5)'>
    ✅ <GoTo '🔹 go to  -> (0, 5, 5)'>
    ✅ <GoTo '🔹 go to  -> (5, 5, 5)'>
    ✅ <GoTo '🔹 go to  -> (5, 0, 5)'>
    ✅ <GoTo '🔹 go to  -> (0, 0, 5)'>
  ✅ <Action '🛬 LAND'>
    ✅ <Land '🔹 land'>


In [ ]:
plan.reset()

In [ ]:
print(plan)

🕓 <Plan '📋 Square Trajectory'>
  🕓 <Action '🔧 PREARM'>
    🕓 <CheckDisarmed '🔹 Check disarmed'>
    🕓 <EFKStatus '🔹 Check EKF status'>
    🕓 <GPSStatus '🔹 Check GPS'>
    🕓 <CheckSystem '🔹 Check system status'>
  🕓 <Action '⚙️  MODE: GUIDED'>
    🕓 <SwitchMode '🔹 Switch to GUIDED'>
  🕓 <Action '🔐 ARM'>
    🕓 <Arm '🔹 arm'>
  🕓 <Action '🛫 TAKEOFF'>
    🕓 <TakeOff '🔹 take off to 1.0 m'>
  🕓 <Action '🛩️ FLY'>
    🕓 <GoTo '🔹 go to  -> (0, 0, 5)'>
    🕓 <GoTo '🔹 go to  -> (0, 5, 5)'>
    🕓 <GoTo '🔹 go to  -> (5, 5, 5)'>
    🕓 <GoTo '🔹 go to  -> (5, 0, 5)'>
    🕓 <GoTo '🔹 go to  -> (0, 0, 5)'>
  🕓 <Action '🛬 LAND'>
    🕓 <Land '🔹 land'>
